In [1]:
import bs4 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from multiprocessing.pool import ThreadPool
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException
import os
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import pickle
import urllib
sns.set()

In [2]:
ABS_PATH = 'trip_advisor_data_restaurants/'

In [3]:
from multiprocessing.pool import Pool
from contextlib import closing

In [4]:
from functools import partial

In [5]:
def read_file(path):
    with open(path, 'r') as f:
        return f.read()

In [6]:
def multiprocess_func(main_input, func, additional_inputs=None,
                      gather_func=None, to_split=True, gather_func_args=None,
                      chunk_size=100, n_processes=8):
    if not gather_func_args:
        gather_func_args = []
    if not additional_inputs:
        additional_inputs = []
    if not gather_func:
        gather_func = lambda x: [z for i in x for z in i]
    if to_split:
        splitted = [(main_input[i:i + chunk_size], *additional_inputs) if additional_inputs else main_input[i:i + chunk_size]\
                    for i in range(0, len(main_input), chunk_size)]
    else:
        splitted = [(i, *additional_inputs) if additional_inputs else i for i in main_input]
    with closing(Pool(n_processes)) as p:
        result = list(tqdm(p.imap(func, splitted),
                           total=len(splitted)))
    return gather_func(result, *gather_func_args)

In [7]:
def process_buble(x):
    return float('.'.join(x))

In [8]:
def bs4_parse_reviews(input_tuple):
    page_to_parse, name = input_tuple
    records = []
    for review_page in bs4.BeautifulSoup(page_to_parse).find_all('div', {'class':'ui_column is-9'}):
        try:
            record = {} 
            record['overall_rating'] = process_buble(review_page.find('span', {'class': 'ui_bubble_rating'})\
                                                     .get('class')[1].split('_')[-1])
            record['title'] = review_page.find('span',{'class':'noQuotes'}).text
            record['review'] = review_page.find('p',{'class':'partial_entry'}).text
            record['name'] = name
            records.append(record)
        except:
            pass

        
    return records

In [9]:
def parse_reviews_multiproc(name, abs_path):
    path = os.path.join(abs_path, name)
    records = []
    for path_page in os.listdir(path):
        page = read_file(os.path.join(path,path_page))
        records.extend(bs4_parse_reviews((page, name)))
    return records

In [10]:
partial_parse_reviews_multiproc = partial(parse_reviews_multiproc, abs_path=ABS_PATH)


In [11]:
df = pd.read_csv('links_to_restaurants.csv')

In [12]:
tmp_df = df[(df['parsed']==True) & (df['review_parsed']==False)]

In [13]:
tmp_df.shape

(5305, 7)

In [14]:
reviews = multiprocess_func(tmp_df['name_to_save'].values,
                  func=partial_parse_reviews_multiproc,
                  to_split=False,
                 n_processes=8)

100%|████████████████████████████████████| 5305/5305 [08:53<00:00,  9.94it/s]


In [15]:
reviews = pd.DataFrame(reviews).drop_duplicates(subset=['name', 'review', 'overall_rating'])

In [16]:
reviews.to_csv('restaurants_reviews_3.csv', index=False, mode='w')

In [17]:
df.loc[df['parsed']==True, 'review_parsed'] = True

In [18]:
df.to_csv('links_to_restaurants.csv', index= False)

In [19]:
df['parsed'].value_counts()

True     7228
False     169
Name: parsed, dtype: int64